In [1]:
import math
import pandas as pd
import numpy as np
import statistics

In [25]:
principal = 500000
rate = 0.05/12
n = 24
data = pd.read_csv("MORTGAGE30US.csv")
vol =statistics.stdev(data['MORTGAGE30US'])*math.sqrt(12)/100
vol

0.1114034576714194

In [16]:
data

,observation_date,MORTGAGE30US
0,1971-04-02,7.33
1,1971-04-09,7.31
2,1971-04-16,7.31
3,1971-04-23,7.31
4,1971-04-30,7.29
...,...,...
2812,2025-02-20,6.85
2813,2025-02-27,6.76
2814,2025-03-06,6.63
2815,2025-03-13,6.65


In [17]:
A = principal * rate * (1+rate)**n/((1+rate)**n-1)
A

42803.74089423373

In [21]:
#for T_10, 1 = stepsTaken, 0 = place
#for T_(n-1)1, n-1 = stepsTaken, 1 = place
#find root.data such that root.data = P

count = 0
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.parent = None
        self.level = None
        self.place = None
def makeTree(steps,A,P,vol):
    global count
    totalcount = count
    rates = np.ones(steps)*rate-(np.array(range(1,steps+1))*rate/100)
    root = Tree()
    root.level = 0
    root.place = 0
    while root.data == None or abs(root.data - P) >= 0.001:
        #recursively make non Callable Tree
        attemptTree2(steps,0,0,root,A,rates,vol)
        diff = abs(root.data - P)
        if diff >= 0.001:
            rates = guessRates(steps,rates,P,root.data,root)
        totalcount+=count
        count = 0
    return (root,rates)
            
def attemptTree(steps, stepsTaken, place, parent, A, rates, vol):    
    nextStep = stepsTaken+1
    global count
    count+=1
    
    if stepsTaken == steps-1:
        placeRate = (1+rates[stepsTaken]*math.exp(place*2*vol*math.sqrt(1/12)))      
        parent.data = A/placeRate
    else:
        placeRate = (1+rates[nextStep]*math.exp(place*2*vol*math.sqrt(1/12)))
        parent.left = Tree()
        parent.right = Tree()
        lower = parent.left
        lower.parent = parent
        upper = parent.right
        upper.parent = parent
        attemptTree(steps, nextStep, place, lower, A, rates, vol)
        attemptTree(steps, nextStep, place+1, upper, A, rates, vol)
        average = (lower.data+upper.data)/2
        parent.data = 1/placeRate*(average+A)

def attemptTree2(steps, stepsTaken, place, parent, A, rates, vol):
    nextStep = stepsTaken+1
    global count
    count+=1
    
    if stepsTaken == steps-1:
        placeRate = (1+rates[stepsTaken]*math.exp(place*2*vol*math.sqrt(1/12)))      
        parent.data = A/placeRate
    else:
        placeRate = (1+rates[nextStep]*math.exp(place*2*vol*math.sqrt(1/12)))
        parent.left = Tree()
        lower = parent.left
        lower.parent = parent
        lower.level = nextStep
        lower.place = place
        if lower.data == None:
            #print(f'lower T_{lower.level},{lower.place}')
            attemptTree2(steps,nextStep,place,lower,A,rates,vol)
            #print(f'lower T_{lower.level},{lower.place}')
        parent.right = Tree()
        upper = parent.right
        upper.parent = parent
        upper.level = nextStep
        upper.place = place+1
        upper.left = lower.right
        if upper.data == None:
            if upper.left == None or nextStep >= steps-1:
                attemptTree2(steps, nextStep, place+1, upper, A, rates, vol)
            else:
                findRight(steps, nextStep, upper.left, place+1, upper, A, rates, vol)
        

        average = (lower.data+upper.data)/2
        parent.data = 1/placeRate*(average+A)
def findRight(steps, stepsTaken, lower, place, parent, A, rates, vol):
    nextStep = stepsTaken+1
    global count
    count+=1
    
    if stepsTaken == steps-1:
        placeRate = (1+rates[stepsTaken]*math.exp(place*2*vol*math.sqrt(1/12)))      
        parent.data = A/placeRate
    else:
        placeRate = (1+rates[nextStep]*math.exp(place*2*vol*math.sqrt(1/12)))
        parent.right = Tree()
        upper = parent.right
        upper.parent = parent
        upper.level = nextStep
        upper.place = place+1
        if lower.level < steps - 1:
            upper.left = lower.right
        if upper.data == None:
            if lower == None or nextStep >= steps-1:
                attemptTree2(steps, nextStep, place+1, upper, A, rates, vol)
            else:
                findRight(steps, nextStep, upper.left, place+1, upper, A, rates, vol)
        

        average = (lower.data+upper.data)/2
        parent.data = 1/placeRate*(average+A)
def guessRates(steps,prevRates,P,base,tree):
    #ri = ri + (T00-P)/(Ti0)*(convergenceVar)
    target = tree
    for step in range(1,steps):
        target = target.left
        diff = base - P
        change = diff/base*prevRates[step]
        new = prevRates[step] + change
        prevRates[step] = new
    return prevRates

In [26]:
count = 0
root,rates = makeTree(n,A,principal,vol)

In [27]:
print(root.data)
print(rates)

500000.0008879544
[0.004125   0.05887562 0.05827485 0.05767408 0.05707331 0.05647253
 0.05587176 0.05527099 0.05467022 0.05406945 0.05346868 0.0528679
 0.05226713 0.05166636 0.05106559 0.05046482 0.04986405 0.04926327
 0.0486625  0.04806173 0.04746096 0.04686019 0.04625942 0.04565864]


In [24]:
parent = root.left
for i in range(n-1):
    print(parent.data,'\t',parent.parent.right.data)
    parent = parent.left

459483.7579479832 	 458743.6746078125
418727.99355391855 	 418120.2432448783
377742.236315731 	 377252.30033428594
336535.82601616735 	 336149.694175981
295117.91066149296 	 294822.079040106
253497.44582381973 	 253278.91375828526
211683.19436396277 	 211529.46075573587
169683.72650752804 	 169582.78549422164
127507.42024807112 	 127447.75629701708
85162.46205230885 	 85133.04452820338
42656.847843501244 	 42647.12509978997


In [9]:
def oldPV(steps,A,P,rate):
    leftover = P
    balances = [P]
    PVs = []
    for step in range(1,steps):
        interest = leftover * rate
        paid = A - interest
        leftover = leftover - paid 
        balances.append(leftover)
        PV = interest/(1+rate)**step
        PVs.append(PV)
    return (PVs,balances)

def refinancePV(steps,start,balance,rates):
    rate = rates[start-1]
    n = steps - start + 1
    A = balance * rate * (1+rate)**n/((1+rate)**n-1)
    total = 0
    leftover = balance
    for step in range(start,steps):
        interest = leftover * rate
        paid = A - interest
        leftover = leftover - paid
        total += interest/(1+rate)
    return total

In [10]:
PVs, balances = oldPV(n,A,principal,rate)
print(balances)
print(A)
for i in range(len(PVs)):
    PVrefi = refinancePV(n,i+2,balances[i+1],rates)
    print(f"Refinance at Month {i+1} refiSavings: ",sum(PVs[i:])-PVrefi)

[500000, 459279.5924390996, 418389.5165133621, 377329.0652712674, 336097.52881566394, 294694.1942914955, 253118.34587347633, 211369.26475371543, 169446.22912928884, 127348.51418976049, 85075.39210465076, 42626.13201085308]
42803.74089423373
Refinance at Month 1 refiSavings:  2691.7570293303907
Refinance at Month 2 refiSavings:  2493.1966144972685
Refinance at Month 3 refiSavings:  2274.3969219920964
Refinance at Month 4 refiSavings:  2039.8553202866215
Refinance at Month 5 refiSavings:  1794.0850495132936
Refinance at Month 6 refiSavings:  1541.616124357954
Refinance at Month 7 refiSavings:  1286.9962489788845
Refinance at Month 8 refiSavings:  1034.7917440001484
Refinance at Month 9 refiSavings:  789.5884856320415
Refinance at Month 10 refiSavings:  555.9928569743391
Refinance at Month 11 refiSavings:  338.63271156280445


In [11]:
print((np.ones(n)/(np.array(range(1,13)))**2))

[1.         0.25       0.11111111 0.0625     0.04       0.02777778
 0.02040816 0.015625   0.01234568 0.01       0.00826446 0.00694444]
